In [ ]:
import os
import json
import pandas as pd
import pathlib
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
import tensorflow as tf

'\nCreated on Sat May 27 15:33:55 2023\n\n@author: adelmobrunelli\n'

In [ ]:
# Set TensorFlow seed
tf.random.set_seed(42)

# Set Numpy seed
np.random.seed(42)

# Set Python seed
random.seed(42)

# Set TensorFlow and Keras seed for reproducibility
tf.random.set_seed(42)

In [ ]:
df = pd.read_csv('/content/drive/My Drive/dataset.csv')
df.columns


# Check if any value is equal to 1
colonne_con_valore_1 = (df == 1).any()

# Selects only columns that have at least one value equal to 1
colonne_selezionate = df.loc[:, colonne_con_valore_1]

# Add the last column of the original DataFrame to the selected columns
colonne_selezionate[df.columns[-1]] = df[df.columns[-1]]

df = colonne_selezionate

print(df.columns)

df = df.iloc[0:6000]

SIZE = 244
X_dataset = []

for i in tqdm(range(df.shape[0])):
    img = image.load_img(df.loc[i, 'ImageID'], target_size=(SIZE, SIZE, 3))
    img = image.img_to_array(img)
    img = img/255.
    X_dataset.append(img)

X = np.array(X_dataset)

y = np.array(df.drop(['ImageID'], axis=1))

# Divide the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Divide the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Print the size of the resulting datasets
print("Dimensioni set di addestramento:", X_train.shape, y_train.shape)
print("Dimensioni set di test:", X_test.shape, y_test.shape)
print("Dimensioni set di validazione:", X_val.shape, y_val.shape)

<ipython-input-6-0e26c418cbf6>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  colonne_selezionate[df.columns[-1]] = df[df.columns[-1]]


Index(['33', '34', '35', '36', '37', '38', '39', '40', '161', '162', '163',
       '164', '165', '167', '168', 'ImageID'],
      dtype='object')


100%|██████████| 3000/3000 [09:03<00:00,  5.52it/s]


Dimensioni set di addestramento: (1920, 244, 244, 3) (1920, 15)
Dimensioni set di test: (600, 244, 244, 3) (600, 15)
Dimensioni set di validazione: (480, 244, 244, 3) (480, 15)



Experiment 1: one dense layer with 256 neurons and one dropout

We try a simple model with a single dense layer of 256 neurons and dropout to fight the high overfitting.


In [ ]:
def lossAndAccuracy():
    #plot the training and validation accuracy and loss at each epoch
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

def precisionAndRecall():
    columnValues = [0, 1, 17, 33, 161, 34, 162, 35, 163, 36, 164, 37, 165, 38, 166,
                    39, 167, 40, 168, 49, 50, 65, 66, 67, 81, 82, 83, 84, 85, 86, 97,
                    98, 99, 100, 113, 255]

    y_pred = model.predict(X_test)

    threshold = 0.5
    y_pred_binary = (y_pred > threshold).astype(int)

    from sklearn.metrics import precision_score, recall_score
    precision = precision_score(y_test, y_pred_binary, average='micro')
    recall = recall_score(y_test, y_pred_binary, average='micro')

def checkLabelsItem(row_index):
    # Seleziona la riga e identifica gli indici dei valori a 1
    indices = df.loc[row_index, df.loc[row_index] == 1].index

    # Stampa gli indici dei valori a 1
    print("Valori a 1 nella riga", 1, ":")
    print(indices)


from sklearn.metrics import roc_curve, auc, accuracy_score

def show_precision_and_roc(model, test_data, test_labels):
    """
    Mostra la precisione e la curva ROC di una rete convoluzionale.

    Parametri:
    model: Il modello di rete convoluzionale già addestrato.
    test_data: Dati di test per valutare il modello.
    test_labels: Etichette corrispondenti ai dati di test.

    """
    # Calcola le predizioni del modello sui dati di test
    predictions = model.predict(test_data)

    # Calcola la precisione
    accuracy = accuracy_score(test_labels.argmax(axis=1), predictions.argmax(axis=1))
    print(f"Precisione: {accuracy:.2f}")

    # Calcola la curva ROC per ogni classe
    num_classes = test_labels.shape[1]
    fpr = {}
    tpr = {}
    roc_auc = {}

    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(test_labels[:, i], predictions[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Calcola la media delle curve ROC per ottenere la curva ROC media
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(num_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= num_classes

    # Calcola l'AUC della curva ROC media
    mean_auc = auc(all_fpr, mean_tpr)

    # Mostra i risultati
    plt.figure()
    plt.plot(all_fpr, mean_tpr, color='b', label=f'Media ROC (area = {mean_auc:.2f})')
    for i in range(num_classes):
        plt.plot(fpr[i], tpr[i], lw=1, label=f'Classe {i} (area = {roc_auc[i]:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Curva ROC')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()


from sklearn.metrics import f1_score

def calculate_f1_score(model, test_data, test_labels):
    """
    Calcola l'F1-score di una rete convoluzionale per una classificazione multi-classe.

    Parametri:
    model: Il modello di rete convoluzionale già addestrato.
    test_data: Dati di test per valutare il modello.
    test_labels: Etichette corrispondenti ai dati di test.

    Restituisce:
    f1_score: L'F1-score calcolato.
    """

    # Calcola le predizioni del modello sui dati di test
    predictions = model.predict(test_data)

    # Calcola l'F1-score
    f1_score_value = f1_score(test_labels.argmax(axis=1), predictions.argmax(axis=1), average='weighted')

    return f1_score_value

In [ ]:
"""
Experiment 1:  one dense layer with 256 neurons

"""
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(SIZE, SIZE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(15, activation='sigmoid'))

model.summary()

model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    epochs=25,
                    batch_size=32,
                    validation_data=(X_val, y_val))

lossAndAccuracy()
show_precision_and_roc(model, X_test, y_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 240, 240, 16)      1216      
                                                                 
 batch_normalization (Batch  (None, 240, 240, 16)      64        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 120, 120, 16)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 120, 120, 16)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 116, 116, 32)      12832     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 58, 58, 32)        0

In [ ]:
"""
Experiment 2:  two dense layer 256/128 neurons and two dropout
"""
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(SIZE, SIZE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(15, activation='sigmoid'))

model.summary()

model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    epochs=25,
                    batch_size=32,
                    validation_data=(X_val, y_val))


lossAndAccuracy()
show_precision_and_roc(model, X_test, y_test)

y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred.argmax(axis=1), average='weighted')

In [ ]:
"""
Experiment 3:  one dense layer and two dropout

"""

from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(SIZE, SIZE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(15, activation='sigmoid'))

model.summary()

model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    epochs=25,
                    batch_size=32,
                    validation_data=(X_val, y_val))

lossAndAccuracy()
show_precision_and_roc(model, X_test, y_test)

In [ ]:
"""
Experiment 4: two dense layer with 256 neurons
"""

from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(SIZE, SIZE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))

model.add(Dense(15, activation='sigmoid'))

model.summary()

model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    epochs=25,
                    batch_size=32,
                    validation_data=(X_val, y_val))


lossAndAccuracy()
show_precision_and_roc(model, X_test, y_test)

y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred.argmax(axis=1), average='weighted')


print(f"F1-Score: {f1:.2f}")

In [ ]:
"""
Experiment 5: two dense layer with 128/256 neurons
"""

from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(SIZE, SIZE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))

model.add(Dense(15, activation='sigmoid'))

model.summary()

model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    epochs=25,
                    batch_size=32,
                    validation_data=(X_val, y_val))


lossAndAccuracy()
show_precision_and_roc(model, X_test, y_test)

y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred.argmax(axis=1), average='weighted')


print(f"F1-Score: {f1:.2f}")